In [1]:
import sys

In [2]:
sys.path.append('/lustre/project/Stat/s1155202250/fastfolder/code/st/istar/')

In [3]:
from extract_features import *

/stat_Lin_Group/s1155202250/basic/conda/envs/pytorchlin/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/stat_Lin_Group/s1155202250/basic/conda/envs/pytorchlin/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
def get_args():
    class Args:
        prefix = '/lustre/project/Stat/s1155202250/fastfolder/code/st/istar/data/10xdata/Xenium_FFPE_Human_Breast_Cancer_Rep1/istar_/'
        device = 'cuda'
        reduction_method = None
        n_components = None
        smoothen_method = 'cv'
        random_weights = False
        use_cache = False
        no_shift = False
        plot = False

    args = Args()
    return args

In [8]:
args = get_args()

In [18]:
wsi = get_data(prefix=args.prefix)

Image loaded from /lustre/project/Stat/s1155202250/fastfolder/code/st/istar/data/10xdata/Xenium_FFPE_Human_Breast_Cancer_Rep1/istar_/he.jpg


In [19]:
wsi.shape

(12288, 15616, 3)

In [20]:
def get_embeddings_shift(
        img, margin=256, stride=64,
        pretrained=True, device='cuda'):
    # margin: margin for shifting. Divisble by 256
    # stride: stride for shifting. Divides `margin`.
    factor = 16  # scaling factor between cls and sub. Fixed
    shape_emb = np.array(img.shape[:2]) // factor
    chans_cls = [
            np.zeros(shape_emb, dtype=np.float32)
            for __ in range(192)]
    chans_sub = [
            np.zeros(shape_emb, dtype=np.float32)
            for __ in range(384)]
    start_list = list(range(0, margin, stride))
    n_reps = 0
    for start0 in start_list:
        for start1 in start_list:
            print(f'shift {start0}/{margin}, {start1}/{margin}')
            t0 = time()
            stop0, stop1 = -margin+start0, -margin+start1
            im = img[start0:stop0, start1:stop1]
            cls, sub = get_embeddings(
                    im, pretrained=pretrained, device=device)
            del im
            # cls, sub: (192, H, W), (384, H, W)  H:752, W:960

            sta0, sta1 = start0 // factor, start1 // factor
            sto0, sto1 = stop0 // factor, stop1 // factor
            for i in range(len(chans_cls)):
                chans_cls[i][sta0:sto0, sta1:sto1] += cls[i]
            del cls
            for i in range(len(chans_sub)):
                chans_sub[i][sta0:sto0, sta1:sto1] += sub[i]
            del sub
            n_reps += 1
            print(int(time() - t0), 'sec')

    mar = margin // factor
    for chan in chans_cls:
        chan /= n_reps
        chan[-mar:] = 0.0
        chan[:, -mar:] = 0.0
    for chan in chans_sub:
        chan /= n_reps
        chan[-mar:] = 0.0
        chan[:, -mar:] = 0.0

    return chans_cls, chans_sub



In [22]:
emb_cls, emb_sub = get_embeddings_shift(
                    wsi, pretrained=(not args.random_weights),
                    device=args.device)

shift 0/256, 0/256
Extracting embeddings...
tile 0 / 12
tile 10 / 12
2801 sec
2801 sec
shift 0/256, 64/256
Extracting embeddings...
tile 0 / 12


In [14]:
emb_cls.__len__()

192

In [15]:
emb_cls[0].shape

(768, 976)

In [16]:
emb_sub.__len__()

384

In [17]:
emb_sub[0].shape

(768, 976)

In [9]:
img=wsi
margin=256
stride=64
pretrained=True
device=args.device



In [10]:
    factor = 16  # scaling factor between cls and sub. Fixed
    shape_emb = np.array(img.shape[:2]) // factor
    chans_cls = [
            np.zeros(shape_emb, dtype=np.float32)
            for __ in range(192)]
    chans_sub = [
            np.zeros(shape_emb, dtype=np.float32)
            for __ in range(384)]
    start_list = list(range(0, margin, stride))
    n_reps = 0

In [11]:
shape_emb

array([528, 704])

In [12]:
start_list

[0, 64, 128, 192]

In [13]:
for start0 in start_list:
    for start1 in start_list:
        print(f'shift {start0}/{margin}, {start1}/{margin}')
        t0 = time()
        stop0, stop1 = -margin+start0, -margin+start1
        im = img[start0:stop0, start1:stop1]
        cls, sub = get_embeddings(
                im, pretrained=pretrained, device=device)
        del im
        sta0, sta1 = start0 // factor, start1 // factor
        sto0, sto1 = stop0 // factor, stop1 // factor
        for i in range(len(chans_cls)):
            chans_cls[i][sta0:sto0, sta1:sto1] += cls[i]
        del cls
        for i in range(len(chans_sub)):
            chans_sub[i][sta0:sto0, sta1:sto1] += sub[i]
        del sub
        n_reps += 1
        print(int(time() - t0), 'sec')


shift 0/256, 0/256
Extracting embeddings...
tile 0 / 6
12 sec
12 sec
shift 0/256, 64/256
Extracting embeddings...
tile 0 / 6
12 sec
12 sec
shift 0/256, 128/256
Extracting embeddings...
tile 0 / 6
12 sec
12 sec
shift 0/256, 192/256
Extracting embeddings...
tile 0 / 6
12 sec
12 sec
shift 64/256, 0/256
Extracting embeddings...
tile 0 / 6
12 sec
13 sec
shift 64/256, 64/256
Extracting embeddings...
tile 0 / 6
12 sec
12 sec
shift 64/256, 128/256
Extracting embeddings...
tile 0 / 6
12 sec
12 sec
shift 64/256, 192/256
Extracting embeddings...
tile 0 / 6
12 sec
12 sec
shift 128/256, 0/256
Extracting embeddings...
tile 0 / 6
12 sec
12 sec
shift 128/256, 64/256
Extracting embeddings...
tile 0 / 6
12 sec
12 sec
shift 128/256, 128/256
Extracting embeddings...
tile 0 / 6
11 sec
12 sec
shift 128/256, 192/256
Extracting embeddings...
tile 0 / 6
12 sec
12 sec
shift 192/256, 0/256
Extracting embeddings...
tile 0 / 6
12 sec
12 sec
shift 192/256, 64/256
Extracting embeddings...
tile 0 / 6
12 sec
12 sec
sh

In [15]:

    mar = margin // factor
    for chan in chans_cls:
        chan /= n_reps
        chan[-mar:] = 0.0
        chan[:, -mar:] = 0.0
    for chan in chans_sub:
        chan /= n_reps
        chan[-mar:] = 0.0
        chan[:, -mar:] = 0.0

In [16]:
        embs = dict(cls=emb_cls, sub=emb_sub)


In [17]:
    embs['rgb'] = np.stack([
            reduce(
                wsi[..., i].astype(np.float16) / 255.0,
                '(h1 h) (w1 w) -> h1 w1', 'mean',
                h=16, w=16).astype(np.float32)
            for i in range(3)])
    del wsi